<a href="https://colab.research.google.com/github/tarachandaniheena/LogisticRegression/blob/main/customer_reviews_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00


In [4]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.0 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

# **Instantiate Model**

In [7]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [8]:
tokens = tokenizer.encode('This is amazing', return_tensors='pt')
tokens[0]

tensor([  101, 10372, 10127, 39854,   102])

In [9]:
result = model(tokens)

In [10]:
int(torch.argmax(result.logits))+1

5

In [13]:
import pandas as pd
import numpy as np
data = pd.read_csv('commerce_reviews.csv')

In [14]:
df = pd.DataFrame(np.array(data['Review Text']),columns=['review'])
df.head()

,review
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress! it's sooo pretty. i happene...
2,I had such high hopes for this dress and reall...
3,"I love, love, love this jumpsuit. it's fun, fl..."
4,This shirt is very flattering to all due to th...


In [15]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1


In [18]:
df_limited['sentiment'] = df_limited['review'].apply(lambda x:sentiment_score(str(x)[:512]))

<ipython-input-18-555a6b851b6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limited['sentiment'] = df_limited['review'].apply(lambda x:sentiment_score(str(x)[:512]))


In [17]:
df_limited = df.head(10000)


In [24]:
df_limited.tail(10)

,review,sentiment
490,This blouse is so pretty and well made. it wil...,5
491,I love the layered look of this jacket. i got ...,5
492,"I really wanted to love this dress, but it was...",2
493,NaN,2
494,I ordered size small and medium because i can ...,4
495,...there's enough room for a small family to l...,3
496,This top is so pretty and feminine. the differ...,5
497,"Very cute, very comfortable. for me aesthetics...",5
498,Cropped and wide- would look cuter on someone ...,2
499,The fun colors drew me to this but it sure fit...,3


In [26]:
df_limited['review'].iloc[72]

"I have a short torso and this works well for me. 34c, bought the 0. there's not much stretch to the fabric so it is fitted to my chest, but not in an uncomfortable way. definitely doesn't hang and have extra fabric like on the model. \r\n\r\nzipper goes almost all the way down to the bottom so it's easier to get on and off which makes up for the lack of stretch n the fabric.\r\n\r\nunlike another reviewer, i found it went really well with navy pants and i wore it to a business meeting under a blazer. wi"

In [30]:
df_limited.to_csv('customer_review_transformer_results2.csv')

Compare results with sentiment.polarity
https://docs.google.com/spreadsheets/d/1y5mLbrDEDQvfhO4fV12Xna7LCwLG_w29qPMXvBVdiXA/edit#gid=2126323537
